
# Day 15 AoC

🕎 [Day 15 description](https://adventofcode.com/2022/day/15) 🕎


## Setup

In [ ]:
# imports
import os, re, sys, IPython, itertools, operator, functools, datetime, heapq

starttime = datetime.datetime.now()

In [ ]:
# common helper, data import
def ans(val):
    return IPython.display.Markdown("**Answer: {}**".format(val))

data_fd = open('inputs/input-aoc-22-15.txt', 'r')
data = data_fd.read().strip().split('\n')

In [ ]:
data[0]

In [ ]:
def make_sbpairs(data):
    sbpairs = []
    xyre = re.compile(r'x=(-?\d+), y=(-?\d+)')
    for line in data:
        sstr, bstr = line.split(":")
        smatch = xyre.search(sstr).groups()
        scoord = (int(smatch[0]), int(smatch[1]))
        bmatch = xyre.search(bstr).groups()
        bcoord = (int(bmatch[0]), int(bmatch[1]))
        sbpairs.append((scoord, bcoord))
    return sbpairs

sbpairs = make_sbpairs(data)
              

## Part 1

In [ ]:
def coverage_at_line(sbpair, lineno):
    sensor, beacon = sbpair
    dis = distance(sbpair)
    ydist = abs(sensor[1] - lineno)
    if ydist > dis:
        return None
    elif ydist == dis:
        return [sensor[0]]
    else:
        extra = dis - ydist
        return list(range(sensor[0]-extra,sensor[0]+extra+1))
    
    
def distance(sbpair):
    sensor, beacon = sbpair
    return abs(sensor[0]-beacon[0]) + abs(sensor[1]-beacon[1])


In [ ]:
def calc_beacon_ex(sbpairs, lineno):
    beacon_exclusion = set()
    existing_beacons = set([sbpair[1][0] for sbpair in sbpairs if sbpair[1][1] ==lineno])
    for sbpair in sbpairs:
        coverage = coverage_at_line(sbpair, lineno)
        if coverage:
            coverage = [ x for x in coverage if x not in existing_beacons ]
            beacon_exclusion.update(coverage)
    return beacon_exclusion

beacon_exclusion = calc_beacon_ex(sbpairs, 2000000)
ans(len(beacon_exclusion))

## Part 2

In [ ]:
def calc_signal_radius(sbpair):
    # returns (sensorx, sensory, r)
    sensor, beacon = sbpair
    return (sensor[0], sensor[1], distance(sbpair))

In [ ]:
sigrad = [ calc_signal_radius(x) for x in sbpairs ]

In [ ]:
def inrad(pt, sigrad):
    sen = sigrad[0],sigrad[1]
    dis = distance((sen, pt))
    if dis <= sigrad[2]:
        extra = sigrad[2]-dis
        return True, extra
    return False, 0

In [ ]:
def check_range(minrange, maxrange, sigrad):
    """
    Walk through each x,y pair, jump ahead a known safe
    amount if in the middle of a range.
    """
    x = minrange
    y = minrange
    while x < maxrange:
        if x%1000000 == 0: print("x:{}".format(x))
        y = minrange
        while y < maxrange:
            coverage_found = False
            maxextra = 0
            for sr in sigrad:
                ir, extra = inrad((x,y), sr)
                if ir:
                    coverage_found = True
                    if extra > maxextra:
                        maxextra = extra
            if not coverage_found:
                v = x*4000000 + y
                print(v)
                return v
            else:
                y+=max(maxextra,1)
        x+=1

def coverage_range(xline, sigrad, ymin,ymax):
    """
    Assume we're going to cover the ymin, ymax range in most cases, go through
    the work of creating an actual list-of-ranges.
    """
    ranges = []
    for si in sigrad:
        xdis = abs(si[0] - xline)
        if si[2] > xdis:
            extra = si[2] - xdis
            ranges.append((si[1]-extra, si[1]+extra))
        elif si[2] == xdis:
            ranges.append((si[1],si[1]))
    ranges.sort(reverse=True)
    merged_ranges = [ranges.pop()]  
    while len(ranges) > 0:
        range = ranges.pop()
        toprange = merged_ranges[-1]
        if range[0] <= toprange[1] and range[1] > toprange[1]:
            merged_ranges[-1] = (toprange[0],range[1])
        elif range[0] < toprange[0]:
            print("bad assumption")
        elif range[0] > toprange[1]:
            merged_ranges.append(range)
    for mr in merged_ranges:
        if mr[0] <= ymin and mr[1] >= ymax:
            return True, mr
    return False, merged_ranges
                
def check_range2(minrange, maxrange, sigrad):
    """
    Walk through the xrange looking for a line that isn't covered. When you find it assume
    that it is a two-item set of ranges and the solution is item[0][1]+1. This breaks if, say,
    there is a small range way outside the search area with no overlap.
    """
    for x in range(minrange, maxrange):
        covered, mr = coverage_range(x, sigrad, minrange, maxrange)
        if not covered:
            print("Hole in line {}".format(x))
            print(mr)
            y = mr[0][1]+1
            v = (x*4000000) + y
            return v

In [ ]:
v = check_range2(0, 4000001, sigrad)
ans(v)

In [ ]:
#v = check_range(0, 4000001, sigrad)
#ans(v)

In [ ]:
endtime = datetime.datetime.now()

print(endtime - starttime)

## Notes


Naive solution takes about 6 minutes (`check_range`), revision that actually calculates ranges, ~20 seconds (`check_range2`).

## Bugs



1. Did not exclude beacons that existed in the count
2. was not returning the right set when calculating exclusion

In [ ]:
testinp="""Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""

testpairs = make_sbpairs(testinp.split('\n'))

In [ ]:
len(calc_beacon_ex(testpairs, 10))